In [248]:
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

required_packages = [
    "pandas", 
    "scikit-learn",  # for the calsulation of the Brier Score
    "scipy",         # for the geometrics means
    "numpy"
]

for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        install_package(package)

In [207]:
import pandas as pd

qa_df = pd.read_csv('rct-a-questions-answers.csv')
daily_forecasts_df = pd.read_csv('rct-a-daily-forecasts.csv')
prediction_sets_df = pd.read_csv('rct-a-prediction-sets.csv')

In [208]:
print("Number of Competition questions, Questions and Answers:", qa_df['discover question id'].nunique())
print("Number of Competition questions, Daily Forecasts:", daily_forecasts_df['discover question id'].nunique())
print("Number of Competition questions, Prediction Sets:", prediction_sets_df['discover question id'].nunique())

Number of Competition questions, Questions and Answers: 343
Number of Competition questions, Daily Forecasts: 187
Number of Competition questions, Prediction Sets: 193


In [209]:
print(qa_df['question name'].unique()[:15])

['How many battle deaths will ACLED record in Sudan in August 2017?'
 "Will ACLED record any riot/protest events in the Cote d'Ivoire (Ivory Coast) during August 2017?"
 'Will FEWS NET publish a Food Security Alert with "famine" and "Yemen" in its headline between 9 August 2017 and 25 October 2017?'
 'Will the Political Instability Task Force (PITF) Worldwide Atrocities Dataset record an event perpetrated by a non-state actor in Israel that starts between 9 August 2017 and 31 August 2017, inclusive?'
 "How many 'hacking or malware (HACK)' data breaches will Privacy Rights Clearinghouse record for September 2017?"
 'How much crude oil will Libya produce in September 2017?'
 'What will the short-term interest rate be for Ireland (IRL) in August 2017?'
 'What will the monthly percent change in the consumer price index (CPI) be for Estonia in August 2017?'
 'How many UN member states will sign the Treaty on the Prohibition of Nuclear Weapons before 1 November 2017?'
 'What will the long-te

In [210]:
print("Number of variables, Questions dataset:", qa_df.shape[1])
qa_df.columns

Number of variables, Questions dataset: 38


Index(['discover question id', 'question name', 'creator id',
       'question starts at', 'question ends at', 'question description',
       'question status', 'question published at', 'question resolved at',
       'question correctness known at', 'use ordinal scoring',
       'question created at', 'question tags', 'question challenges',
       'discover answer id', 'answer name', 'answer initial probability',
       'answer sort order', 'answer resolved at', 'answer resolved by user id',
       'answer resolved probability', 'answer correctness known at',
       'answer created at', 'clarification 1', 'clarification 2',
       'clarification 3', 'IFP Generation Method', 'Country - Primary',
       'Country - Secondary', 'Topic', 'Region',
       'Non-state violence (Terrorism)', 'Domain', 'Created By', 'Deaths',
       'RCT', 'Controversial',
       'Departure from Status Quo Resolution (Binary Only)'],
      dtype='object')

In [211]:
answer_counts = qa_df.groupby('discover question id')['discover answer id'].count()
print("Median number of answers submitted per question:", answer_counts.median())
print("Min. number of answers per question:", answer_counts.min())
print("Max. number of answers per question:", answer_counts.max())

Median number of answers submitted per question: 3.0
Min. number of answers per question: 1
Max. number of answers per question: 5


The "rct-a-questions-answers" dataset contains 343 forecasting questions posed to participants, along with their corresponding answers. It includes variables that describe the topics of the questions, their contexts, and the criteria for resolution. Additionally, the dataset includes IDs for both topics and respondents, timestamps for when the questions were published and resolved, geographical region attributions, probabilities of predicted outcomes with some other charasterisrics. The number of answers submitted to each question varies, with a median of 3 answers per question, and a range of 1 to 5 answers.

In [212]:
print("Daily forecasts dataset structure:", daily_forecasts_df.shape)
daily_forecasts_df.columns

Daily forecasts dataset structure: (9251564, 11)


Index(['date', 'discover question id', 'discover answer id', 'site id',
       'external predictor id', 'question id', 'answer id',
       'external prediction id', 'external prediction set id', 'created at',
       'forecast'],
      dtype='object')

In [213]:
daily_forecasts_df.head()

,date,discover question id,discover answer id,site id,external predictor id,question id,answer id,external prediction id,external prediction set id,created at,forecast
0,2018-03-07T14:00:59-05:00,177,561,5,78,820,2471,6892,2359,2018-03-07T18:31:21Z,0.2
1,2018-03-07T14:00:59-05:00,177,559,5,77,820,2473,6885,2358,2018-03-07T18:31:21Z,0.2
2,2018-03-07T14:00:59-05:00,177,558,5,65,820,2474,6824,2346,2018-03-07T18:31:20Z,0.2
3,2018-03-07T14:00:59-05:00,177,558,5,76,820,2474,6879,2357,2018-03-07T18:31:21Z,0.2
4,2018-03-07T14:00:59-05:00,177,559,5,57,820,2473,6785,2338,2018-03-07T18:31:18Z,0.2


In [214]:
print("Number of participating entities:", daily_forecasts_df['site id'].nunique())
print("Total number of models or methods used by participants:", daily_forecasts_df['external predictor id'].nunique())

print(f"Total number of forecast entries: {daily_forecasts_df.shape[0]}")



Number of participating entities: 7
Total number of models or methods used by participants: 320
Total number of forecast entries: 9251564


"rct-a-daily-forecasts" dataset contains the most recent predictions submitted by various forecasting methods or models from 7 participating entities.
Each forecast is associated with a specific question ("question id") and an answer option ("answer id") within that question.'forecast' values capture the probability of these forecasted outcomes.

In [215]:
prediction_sets_df[:5]

,prediction set id,membership guid,discover question id,question id,question name,prediction set created at,prediction set updated at,rationale,comment id,prediction id,...,answer resolved probability,answer sort order,question ends_at,question published_at,question starts_at,question resolved_at,question correctness_known_at,question use_ordinal_scoring,site id,site name
0,340,52d0b6592281f0d274d93661cf2ef1dd26f55d28,182,807,Practice Question: Will there be a new prime m...,2018-02-28T20:16:37Z,2018-02-28T20:16:37Z,Given that the negotiation of Brexit is still ...,391,704,...,NaN,0,2018-03-06T19:00:41Z,2018-02-26T17:05:25Z,2018-02-26T16:49:41Z,NaN,NaN,False,11,HFC Carbon
1,341,52d0b6592281f0d274d93661cf2ef1dd26f55d28,183,816,Practice Question: What will be the daily clos...,2018-02-28T20:18:42Z,2018-02-28T20:18:42Z,NaN,392,705,...,NaN,0,2018-03-11T18:00:43Z,2018-02-26T17:05:27Z,2018-02-26T17:00:01Z,NaN,NaN,True,11,HFC Carbon
2,341,52d0b6592281f0d274d93661cf2ef1dd26f55d28,183,816,Practice Question: What will be the daily clos...,2018-02-28T20:18:42Z,2018-02-28T20:18:42Z,NaN,392,706,...,NaN,1,2018-03-11T18:00:43Z,2018-02-26T17:05:27Z,2018-02-26T17:00:01Z,NaN,NaN,True,11,HFC Carbon
3,341,52d0b6592281f0d274d93661cf2ef1dd26f55d28,183,816,Practice Question: What will be the daily clos...,2018-02-28T20:18:42Z,2018-02-28T20:18:42Z,NaN,392,707,...,NaN,2,2018-03-11T18:00:43Z,2018-02-26T17:05:27Z,2018-02-26T17:00:01Z,NaN,NaN,True,11,HFC Carbon
4,341,52d0b6592281f0d274d93661cf2ef1dd26f55d28,183,816,Practice Question: What will be the daily clos...,2018-02-28T20:18:42Z,2018-02-28T20:18:42Z,NaN,392,708,...,NaN,3,2018-03-11T18:00:43Z,2018-02-26T17:05:27Z,2018-02-26T17:00:01Z,NaN,NaN,True,11,HFC Carbon


In [216]:
print("Number of individual forecasters:", prediction_sets_df['membership guid'].nunique())

Number of individual forecasters: 5062


The "rct-a-prediction-sets" dataset contains detailed records of individual-level forecasts submitted by competing entities. In total it contains the forecasts from 5062 individual forecasters, each identified by a unique membership GUID. The 'rationale' variable contains the explanations given by the forecasters in support to their prediction.

In [217]:
prediction_sets_df['filled at'] = pd.to_datetime(prediction_sets_df['filled at'])

prediction_sets_df = prediction_sets_df.sort_values(by=['membership guid', 'discover question id', 'answer id', 'prediction set id', 'filled at'],
                                                    ascending=[True, True, True, True, False])

# create a column with the date of the forecast, without the exact time
prediction_sets_df['date'] = prediction_sets_df['filled at'].dt.date

# keeping only the most recent forecast for each forecaster, question, set and answer per every day
individual_daily_forecasts_df = prediction_sets_df.drop_duplicates(
    subset=['membership guid', 'discover question id', 'answer id', 'prediction set id', 'date'],
    keep='first'
)

individual_daily_forecasts_columns = [
    'date',
    'discover question id',
    'membership guid',
    'answer id',
    'forecasted probability'
]

individual_daily_forecasts_df = individual_daily_forecasts_df[individual_daily_forecasts_columns]

In [218]:
# check of the number of day-question pairs
unique_question_day_pairs = prediction_sets_df[['discover question id', 'date']].drop_duplicates().sort_values(by=['discover question id', 'date'], ascending=True)
print("Unique question-day pairs", unique_question_day_pairs.shape[0])

Unique question-day pairs 13139


In [219]:
missing_values = individual_daily_forecasts_df.isnull().sum()
print("Missing values in key columns:", missing_values)

Missing values in key columns: date                      0
discover question id      0
membership guid           0
answer id                 0
forecasted probability    0
dtype: int64


In [220]:
num_duplicates = individual_daily_forecasts_df.duplicated().sum()
print(f"Number of duplicates: {num_duplicates}")

Number of duplicates: 32607


In [221]:
print(individual_daily_forecasts_df.dtypes)

date                       object
discover question id        int64
membership guid            object
answer id                   int64
forecasted probability    float64
dtype: object


# Aggregations

I aggregate the data by:

- Competition questions ("discover question id"): it has a global identifier for each unique competition question across the entire forecasting tournament (unlike the "question id", which varies across competitors)
- Answer ("answer id"): it identifies the unique answers given by the competitors within the range of answers for each question. This ensures that forecasts are aggregated for each potential answer option within the question.
- Date ("date"): this variable is used for aggregation, since we are interested in the most recent forecast per day from every competitor

In [222]:
# 1. Raw mean
# Calculate the simple average of the forecasted probabilities for each question-day-answer
grouped_idf_df = individual_daily_forecasts_df.groupby(['discover question id', 'answer id', 'date'])
# individual_daily_forecasts_df = individual_daily_forecasts_df.groupby(['discover question id', 'date', 'answer id', 'answer name'])

raw_mean_df = grouped_idf_df['forecasted probability'].mean().reset_index(name='raw_mean')

In [223]:
print("Summary statistics of Raw Mean:")
print(raw_mean_df['raw_mean'].describe())

Summary statistics of Raw Mean:
count    157519.000000
mean          0.236800
std           0.264033
min           0.000000
25%           0.025000
50%           0.140000
75%           0.358894
max           1.000000
Name: raw_mean, dtype: float64


In [224]:
# 2. Median
# Calculate the median of the forecasted probabilities for each question-day-answer
median_df = grouped_idf_df['forecasted probability'].median().reset_index(name='median')

In [225]:
print("Summary statistics of Median:")
print(median_df['median'].describe())

Summary statistics of Median:
count    157519.000000
mean          0.227932
std           0.273743
min           0.000000
25%           0.010000
50%           0.110000
75%           0.350000
max           1.000000
Name: median, dtype: float64


In [226]:
# 3. Geometric Mean
# Calculate the geometric mean of the forecasted probabilities for each question-day-answer

from scipy.stats import gmean

geometric_mean_df = grouped_idf_df['forecasted probability'].apply(gmean).reset_index(name='geometric_mean')

In [227]:
print("Summary statistics of Geometric Mean:")
print(geometric_mean_df['geometric_mean'].describe())

Summary statistics of Geometric Mean:
count    157519.000000
mean          0.188625
std           0.266226
min           0.000000
25%           0.000000
50%           0.050000
75%           0.282672
max           1.000000
Name: geometric_mean, dtype: float64


In [228]:
# Trimmed Mean (trimming top and bottom 10% of forecasts)
# Calculate the trimmed mean of the forecasted probabilities for each question-day-answer
# Trimmed mean is calculated by removing the top and bottom 10% of the forecasts and then calculating the mean of the remaining forecasts.

from scipy.stats import trim_mean
trimmed_mean_df = grouped_idf_df['forecasted probability'].apply(lambda x: trim_mean(x, proportiontocut=0.1)).reset_index(name='trimmed_mean')

In [229]:
print("Summary statistics of Trimmed Mean (10 percent cut):")
print(trimmed_mean_df['trimmed_mean'].describe())

Summary statistics of Trimmed Mean (10 percent cut):
count    157519.000000
mean          0.235645
std           0.264799
min           0.000000
25%           0.025000
50%           0.137325
75%           0.356667
max           1.000000
Name: trimmed_mean, dtype: float64


In [230]:
# 5 Geometric Mean of Odds
# Calculate the geometric mean of the odds of the forecasted probabilities for each question-day-answer

# geom_mean_odds = individual_daily_forecasts_df.obj
geom_mean_odds = individual_daily_forecasts_df

geom_mean_odds['odds'] = geom_mean_odds['forecasted probability'] / (1 - geom_mean_odds['forecasted probability'])

In [231]:
print("Summary statistics of Odds:")
print(geom_mean_odds['odds'].describe())

Summary statistics of Odds:
count    7.501810e+05
mean              inf
std               NaN
min      0.000000e+00
25%      1.010101e-02
50%      1.272686e-01
75%      6.393443e-01
max               inf
Name: odds, dtype: float64


c:\Users\Homo Nicetas\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In [232]:

import numpy as np
## since there are infinite values, we need to replace them with a finite large number to mark them as 100% certain event
## the 75% value is 6, so I replace infinite value with 7 as a cutoff of the certain event
geom_mean_odds['odds'].replace([np.inf], 7, inplace=True)

geom_mean_odds = geom_mean_odds.groupby(['discover question id', 'answer id', 'date'])
geom_mean_odds = geom_mean_odds['odds'].apply(gmean).reset_index(name='geom_mean_of_odds')

C:\Users\Homo Nicetas\AppData\Local\Temp\ipykernel_17192\503182684.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  geom_mean_odds['odds'].replace([np.inf], 7, inplace=True)


In [233]:
aggregated_df = raw_mean_df.merge(median_df, on=['discover question id', 'answer id', 'date'])
aggregated_df = aggregated_df.merge(geometric_mean_df, on=['discover question id', 'answer id', 'date'])
aggregated_df = aggregated_df.merge(trimmed_mean_df, on=['discover question id', 'answer id', 'date'])
aggregated_df = aggregated_df.merge(geom_mean_odds, on=['discover question id', 'answer id', 'date'])

# Calculation of the Brier score: How do the aggregation methods perform on the competition data?


In [234]:
print("Questions & Answers dataset, Resolved Probabilities:")
print(qa_df['answer resolved probability'].unique())
print(qa_df['answer resolved probability'].head())

Questions & Answers dataset, Resolved Probabilities:
[ 0.  1. nan]
0    0.0
1    1.0
2    0.0
3    0.0
4    0.0
Name: answer resolved probability, dtype: float64


In [235]:
print(qa_df["discover answer id"])

0        13
1        14
2        15
3        16
4        17
       ... 
950    1105
951    1106
952    1107
953    1108
954    1109
Name: discover answer id, Length: 955, dtype: int64


We need to merge the dataset with calculated aggregations (aggregated_df) with the answers with resolved probabilities from Questions & Answers dataset (qa_df).

In [236]:
answer_id_df = daily_forecasts_df[['discover question id', 'answer id', 'discover answer id']]
answer_id_df = answer_id_df.drop_duplicates()

In [237]:
aggregated_df['discover question id'] = aggregated_df['discover question id'].astype('category')
aggregated_df['answer id'] = aggregated_df['answer id'].astype('category')
for col in aggregated_df.select_dtypes(include=['float64']).columns:
    aggregated_df[col] = aggregated_df[col].astype('float32')

answer_id_df['discover question id'] = answer_id_df['discover question id'].astype('category')
answer_id_df['discover answer id'] = answer_id_df['discover answer id'].astype('category')
answer_id_df['answer id'] = answer_id_df['answer id'].astype('category')

In [238]:
id_aggregated_df = aggregated_df.merge(answer_id_df,
                                on=['discover question id', 'answer id'],
                                how = 'left')

In [239]:
missing_values_count = id_aggregated_df['discover answer id'].isnull().sum()
total_entries_count = len(id_aggregated_df)

print(f"Missing entries in Discover Answer ID after merging: {missing_values_count} out of {total_entries_count} ({missing_values_count / total_entries_count:.2%})")

Missing entries in Discover Answer ID after merging: 463 out of 157519 (0.29%)


In [240]:
id_aggregated_df = id_aggregated_df.dropna(subset=['discover answer id'])
id_aggregated_df[['discover question id', 'discover answer id', 'answer id']] = id_aggregated_df[['discover question id', 'discover answer id', 'answer id']].astype('category')

In [241]:
resolved_probabilities_df = qa_df[['discover question id', 'discover answer id', 'answer name', 'answer resolved probability']]

resolved_probabilities_df['discover question id'] = resolved_probabilities_df['discover question id'].astype('category')
resolved_probabilities_df['discover answer id'] = resolved_probabilities_df['discover answer id'].astype('category')

C:\Users\Homo Nicetas\AppData\Local\Temp\ipykernel_17192\3127621872.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resolved_probabilities_df['discover question id'] = resolved_probabilities_df['discover question id'].astype('category')
C:\Users\Homo Nicetas\AppData\Local\Temp\ipykernel_17192\3127621872.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  resolved_probabilities_df['discover answer id'] = resolved_probabilities_df['discover answer id'].astype('category')


In [242]:
comparison_df = id_aggregated_df.merge(
    resolved_probabilities_df,
    on=['discover question id', 'discover answer id'],
    how='left'
)

comparison_df.head()

,discover question id,answer id,date,raw_mean,median,geometric_mean,trimmed_mean,geom_mean_of_odds,discover answer id,answer name,answer resolved probability
0,177,2466,2018-03-08,0.390000,0.50,0.223885,0.390000,0.416046,561,"Less than 2,000",1.0
1,177,2466,2018-03-09,0.221667,0.20,0.157849,0.221667,0.207689,561,"Less than 2,000",1.0
2,177,2466,2018-03-10,0.033333,0.05,0.000000,0.033333,0.000000,561,"Less than 2,000",1.0
3,177,2466,2018-03-11,0.100000,0.10,0.000000,0.100000,0.000000,561,"Less than 2,000",1.0
4,177,2466,2018-03-12,0.000000,0.00,0.000000,0.000000,0.000000,561,"Less than 2,000",1.0


I calculate the Brier score for each aggregation method.
Since Brier score requires the probabilities for calculation, I convert the Geometric Mean of Odds back to probabilities.

In [243]:
comparison_df['geom_mean_of_odds_prob'] = comparison_df['geom_mean_of_odds'] / (1 + comparison_df['geom_mean_of_odds'])

In [244]:
from sklearn.metrics import brier_score_loss

brier_scores = {}

aggregation_methods = ['raw_mean', 'median', 'geometric_mean', 'trimmed_mean', 'geom_mean_of_odds_prob']

for method in aggregation_methods:
    score = brier_score_loss(comparison_df['answer resolved probability'], comparison_df[method])
    brier_scores[method] = score

for method, score in brier_scores.items():
    print(f"Brier Score for {method}: {score:.4f}")

Brier Score for raw_mean: 0.1286
Brier Score for median: 0.1296
Brier Score for geometric_mean: 0.1344
Brier Score for trimmed_mean: 0.1285
Brier Score for geom_mean_of_odds_prob: 0.1341


According to the calculated Brier scores, the aggregation methods with highest level of performance on this dataset are the *trimmed mean* and the *raw mean*. Both methods aggregate forecasts by taking averages across all question-answer-day combinations, with the key difference being that the trimmed mean excludes the top and bottom 10% of extreme probability values. This exclusion helps mitigate the impact of predictions that might be less reliable or overly optimistic/pessimistic, leading to forecasts that better reflect the central tendency of the data.

The *median* performed slighly less accurately than the raw mean and the trimmed mean, though the difference in the Brier score is very little. The median’s performance is close to the raw mean and trimmed mean but slightly less accurate, likely because it does not consider the full distribution of predictions.

The *geometric mean* and the *geometric mean of odds* exhibited the worst performance compared to the other methods. The geometric mean may perform poorly on this dataset due to a significant number of probabilities being close to zero. Since the geometric mean involves multiplying all values together and then taking the root, a large number of small values (or zeros) can lead to an aggregation that is heavily skewed towards lower values, resulting in more moderate aggregated probabilities compared to methods like the mean and median.

Also, with geometric mean and the geometric mean of odds if any probability within a set for aggregation is zero, then the aggregation will be 0 as well.

In [245]:
individual_daily_forecasts_df.head()

idf_df = individual_daily_forecasts_df

## calculate the variance of the forecasted probabilities across days for each question-answer pair
grouped_idf_df = idf_df.groupby(['discover question id', 'answer id'])

variance_df = grouped_idf_df['forecasted probability'].var().reset_index(name='variance')

## since the variance of groups with 1 observation is automatically filled as NA, I fill the NA values with 0
variance_df['variance'] = variance_df['variance'].fillna(0)

idf_df = idf_df.merge(variance_df, on=['discover question id', 'answer id'])

## calculate the weights of the foresasts based on the variance
idf_df['weights'] = (1 - idf_df['variance']).clip(lower=0)

## weight is assigned as 1 - variance * probability
idf_df['weighted probability'] = idf_df['weights'] * idf_df['forecasted probability']

## calculate the weighted mean of the forecasted probabilities for each question-answer pair
weighted_idfs = idf_df.groupby(['discover question id', 'answer id', 'date'])['weighted probability'].mean().reset_index(name='weighted_forecast')

weighted_comparison = weighted_idfs.merge(answer_id_df, on=['discover question id', 'answer id'])

weighted_comparison = weighted_comparison.merge(qa_df[['discover question id', 'discover answer id', 'answer resolved probability']],
                                    on=['discover question id', 'discover answer id'])

In [246]:
brier_score = brier_score_loss(
    weighted_comparison['answer resolved probability'], 
    weighted_comparison['weighted_forecast']
)

print(f"Brier Score for Weighted Forecast probabilities: {brier_score:.4f}")

Brier Score for weighted forecast: 0.1267


I implemented the forecast aggregation based on the *mean of weighted probabilities*. The weights are derived from the variance of forecast probabilities across days, so this method leverages variance as a proxy for the certainty of forecasts.
In this case the assigned weights were calculated as:

1 - Forecast probability variance across days

The assumption of this method is that forecast probabilities with higher variance over time indicate greater uncertainty. Higher variance implies that the forecasters are less confident in their predictions, so these forecasts should contribute less to the aggregated forecast. Hence, they are also assigned lower weights compared to the foreasts with lower variance across days.

This method also received the lowest Brier score, compared to other methods including those that performed the best on this dataset.